In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [7]:
pip install category_encoders

     ---------------------------------------- 81.9/81.9 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy.stats import gaussian_kde
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor  
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error,f1_score
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import GradientBoostingClassifier

### Область работы 2 (поиск  модели .... )

In [11]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

train = 'train.csv'
df = pd.read_csv(train)
df.drop_duplicates(inplace = True)
df # содержит только имя файла, без имен папок !!!

,C1,N2,N3,C4_enc,C5_enc,C6_enc,N7,C8,C9,N10,C11,C12_enc,N13,N14,Target
0,1.0,28.00,2.000,2.0,4.0,8.0,4.165,1.0,1.0,2.0,1.0,2.0,181.0,1.0,1.0
1,1.0,36.25,5.000,2.0,8.0,5.0,2.500,1.0,1.0,6.0,0.0,2.0,0.0,368.0,1.0
2,0.0,29.50,2.000,1.0,10.0,8.0,2.000,0.0,0.0,0.0,0.0,2.0,256.0,18.0,0.0
3,1.0,22.67,0.165,2.0,8.0,3.0,2.250,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,0.0,25.75,0.500,2.0,8.0,8.0,0.875,1.0,0.0,0.0,1.0,2.0,491.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,1.0,28.75,1.165,2.0,4.0,4.0,0.500,1.0,0.0,0.0,0.0,1.0,280.0,1.0,0.0
582,1.0,44.83,7.000,1.0,8.0,4.0,1.625,0.0,0.0,0.0,0.0,2.0,160.0,3.0,0.0
583,1.0,21.33,7.500,2.0,6.0,4.0,1.415,1.0,1.0,1.0,0.0,2.0,80.0,9801.0,1.0
584,0.0,53.33,0.165,2.0,1.0,1.0,0.000,0.0,0.0,0.0,1.0,1.0,62.0,28.0,0.0


In [12]:
X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)

In [13]:
cat_enc_1 = ['C6_enc','C5_enc','C4_enc','C12_enc']
num_features = ['N2','N3','N7','N10','N13','N14']
cat_features = ['C1','C9','C11']

# проверим метод КНН

In [14]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [15]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [16]:
model_baseline =Pipeline(steps=[
        ('preproc', ct),
        ('estimator', KNeighborsClassifier())
    ],
)
model_baseline.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['N2', 'N3', 'N7', 'N10',
                                                   'N13', 'N14']),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C1', 'C9', 'C11']),
                                                 ('cat_enc_1',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C6_enc', 'C5_enc', 'C4_enc',
                                                   'C12_enc'])])),
                ('estimator', KNeighborsClassifier())])

In [17]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
   'estimator__n_neighbors': [30, 35, 40, 45, 50],
    'estimator__p': [2,3,4],
    'estimator__weights': ['uniform', 'distance']  
}

In [18]:
baseline_grid = GridSearchCV(model_baseline, n_jobs=-1, param_grid=param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['N2',
                                                                          'N3',
                                                                          'N7',
                                                                          'N10',
                                                                          'N13',
                                                                          'N14']),
                                                                        ('cat_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C1',
                                                                          'C9',
                                                                          'C11']),
                                                                        ('cat_enc_1',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C6_enc',
                                                                          'C...
             param_grid={'estimator__n_neighbors': [30, 35, 40, 45, 50],
                         'estimator__p': [2, 3, 4],
                         'estimator__weights': ['uniform', 'distance'],
                         'preproc__cat_enc_1__scaler': [StandardScaler(),
                                                        RobustScaler(),
                                                        MinMaxScaler()],
                         'preproc__cat_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler()],
                         'preproc__num_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler(),
                                                              Normalizer()]},
             scoring='f1')

In [19]:
print("Лучшие параметры:", baseline_grid.best_params_)

Лучшие параметры: {'estimator__n_neighbors': 40, 'estimator__p': 3, 'estimator__weights': 'uniform', 'preproc__cat_enc_1__scaler': MinMaxScaler(), 'preproc__cat_transformer__scaler': RobustScaler(), 'preproc__num_transformer__scaler': MinMaxScaler()}


In [20]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

Метрика F1 модели :  0.8275862068965516


# Проверим метод градиентного бустинга

In [21]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [22]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [23]:
model_baseline_1 = Pipeline(steps=[
        ('preproc', ct),
        ('estimator', GradientBoostingClassifier())
    ],
)
model_baseline_1.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['N2', 'N3', 'N7', 'N10',
                                                   'N13', 'N14']),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C1', 'C9', 'C11']),
                                                 ('cat_enc_1',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C6_enc', 'C5_enc', 'C4_enc',
                                                   'C12_enc'])])),
                ('estimator', GradientBoostingClassifier())])

In [24]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
  # 'estimator__learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7],
  #  'estimator__subsample': [0.3,0.4,0.5,0.6],
  #  'estimator__n_estimators': [300,400,500],
    'estimator__criterion':['friedman_mse','squared_error'],
    'estimator__max_depth': [2,3,4,5]
}

In [25]:
baseline_grid = GridSearchCV(model_baseline_1, n_jobs=-1, param_grid = param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['N2',
                                                                          'N3',
                                                                          'N7',
                                                                          'N10',
                                                                          'N13',
                                                                          'N14']),
                                                                        ('cat_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C1',
                                                                          'C9',
                                                                          'C11']),
                                                                        ('cat_enc_1',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C6_enc',
                                                                          'C...
             param_grid={'estimator__criterion': ['friedman_mse',
                                                  'squared_error'],
                         'estimator__max_depth': [2, 3, 4, 5],
                         'preproc__cat_enc_1__scaler': [StandardScaler(),
                                                        RobustScaler(),
                                                        MinMaxScaler()],
                         'preproc__cat_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler()],
                         'preproc__num_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler(),
                                                              Normalizer()]},
             scoring='f1')

### найдем лучшие параметры, а в дальнейшем некоторые подберем вручную

In [26]:
print("Лучшие параметры:", baseline_grid.best_params_)

Лучшие параметры: {'estimator__criterion': 'squared_error', 'estimator__max_depth': 3, 'preproc__cat_enc_1__scaler': StandardScaler(), 'preproc__cat_transformer__scaler': MinMaxScaler(), 'preproc__num_transformer__scaler': StandardScaler()}


In [27]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

Метрика F1 модели :  0.8363636363636363


In [28]:
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', StandardScaler())    
 ])
gb = Pipeline( steps = [   
('preproc', ct),
('estimator', GradientBoostingClassifier(criterion = 'friedman_mse',
                                         n_estimators = 400,subsample=0.1,learning_rate=0.01,
                                         max_depth =3 ,random_state=42))])
gb.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['N2', 'N3', 'N7', 'N10',
                                                   'N13', 'N14']),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C1', 'C9', 'C11']),
                                                 ('cat_enc_1',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C6_enc', 'C5_enc', 'C4_enc',
                                                   'C12_enc'])])),
                ('estimator',
                 GradientBoostingClassifier(learning_rate=0.01,
                                            n_estimators=400, random_state=42,
                                            subsample=0.1))])

In [29]:
y_pred = gb.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.8648648648648648

In [30]:
y_pred = gb.predict(X_train)
f1 = f1_score(y_train, y_pred)
f1

0.8942307692307693

In [31]:
clf = GradientBoostingClassifier(learning_rate = 0.1, subsample = 0.5, random_state = 42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Расчет метрики F1
f1 = f1_score(y_test, y_pred)
f1

0.8518518518518519

In [32]:
np.mean(cross_val_score(clf, X, y, scoring = 'f1', cv = 100))

0.8401428571428571

# Рандом форест

In [33]:
num_transformer = Pipeline(steps=[
    ('scaler', Normalizer())
])
cat_transformer = Pipeline(steps=[
    ('scaler', Normalizer())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', Normalizer())    
 ])

In [34]:
ct = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, num_features),
      ('cat_transformer', cat_transformer, cat_features),
         ('cat_enc_1',cat_enc_1_transformer , cat_enc_1),
    ],
    remainder='passthrough'
)

In [77]:
model_baseline_2 = Pipeline(steps=[
        ('preproc', ct),
        ('estimator', RandomForestClassifier())
    ],
)
model_baseline_1.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['N2', 'N3', 'N7', 'N10',
                                                   'N13', 'N14']),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C1', 'C9', 'C11']),
                                                 ('cat_enc_1',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C6_enc', 'C5_enc', 'C4_enc',
                                                   'C12_enc'])])),
                ('estimator', GradientBoostingClassifier())])

In [78]:
param_grid_base = {
    'preproc__num_transformer__scaler' :[StandardScaler(), RobustScaler(), MinMaxScaler(), Normalizer()],
     'preproc__cat_transformer__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
     'preproc__cat_enc_1__scaler' :[StandardScaler(), RobustScaler(),MinMaxScaler()],
   'estimator__criterion': ['gini','entropy'],
    #'estimator__n_estimators': range(50,400),
    'estimator__max_features': range(4,11),
    #'estimator__min_samples_split': [2, 5, 10],
    
}

In [79]:
baseline_grid = GridSearchCV(model_baseline_2, n_jobs=-1, param_grid = param_grid_base, scoring='f1')
baseline_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['N2',
                                                                          'N3',
                                                                          'N7',
                                                                          'N10',
                                                                          'N13',
                                                                          'N14']),
                                                                        ('cat_transformer',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C1',
                                                                          'C9',
                                                                          'C11']),
                                                                        ('cat_enc_1',
                                                                         Pipeline(steps=[('scaler',
                                                                                          Normalizer())]),
                                                                         ['C6_enc',
                                                                          'C...
             param_grid={'estimator__criterion': ['gini', 'entropy'],
                         'estimator__max_features': range(4, 11),
                         'preproc__cat_enc_1__scaler': [StandardScaler(),
                                                        RobustScaler(),
                                                        MinMaxScaler()],
                         'preproc__cat_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler()],
                         'preproc__num_transformer__scaler': [StandardScaler(),
                                                              RobustScaler(),
                                                              MinMaxScaler(),
                                                              Normalizer()]},
             scoring='f1')

In [80]:
print("Лучшие параметры:", baseline_grid.best_params_)

Лучшие параметры: {'estimator__criterion': 'gini', 'estimator__max_features': 4, 'preproc__cat_enc_1__scaler': MinMaxScaler(), 'preproc__cat_transformer__scaler': StandardScaler(), 'preproc__num_transformer__scaler': Normalizer()}


In [81]:
y_pred = baseline_grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Метрика F1 модели : ", f1)

Метрика F1 модели :  0.8256880733944955


### Соберем модель и снова добавим несколько параметров вручную

In [82]:
num_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])
cat_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())    
])
cat_enc_1_transformer = Pipeline(steps=[
     ('scaler', RobustScaler())    
 ])
gb = Pipeline( steps = [   
('preproc', ct),
('estimator', RandomForestClassifier(criterion = 'entropy', max_features = 4,
                                    n_estimators = 400,
                                    random_state=42))])
gb.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['N2', 'N3', 'N7', 'N10',
                                                   'N13', 'N14']),
                                                 ('cat_transformer',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C1', 'C9', 'C11']),
                                                 ('cat_enc_1',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['C6_enc', 'C5_enc', 'C4_enc',
                                                   'C12_enc'])])),
                ('estimator',
                 RandomForestClassifier(criterion='entropy', max_features=4,
                                        n_estimators=400, random_state=42))])

In [83]:
forest = RandomForestClassifier(criterion= 'entropy', n_estimators = 59, max_features = 9,
                                random_state = 42)
forest.fit(X_train, y_train)
y_pred_best = forest.predict(X_test)
print('Правильность на тестовом наборе:', f1_score(y_test, y_pred_best))

Правильность на тестовом наборе: 0.875


In [84]:
y_pred = gb.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.8518518518518519

In [85]:
y_pred = gb.predict(X_train)
f1 = f1_score(y_train, y_pred)
f1 #переобучена

1.0

###### Лучшее значение = 0.8545454545454546